In [1]:
import pandas as pd
import re

In [2]:
import os
authors = ["Grimms","HCAndersen","Potter"]
#columns = ['Emotion', 'Text', 'NovelId','SentenceId','emoA','emoB','moodA','moodB',] #2 persons labelled emotions & mood, used interchangeably in this project
columns = ['NovelId','SentenceId', 'Emotion', 'Text'] 
pattern = r'(\S+):(\S+)\s(\S+):(\S+)\s(\S+):(\S+)\s(.*)'
data = []
novel_map = []
NovelId = -1
for author in authors:
    path = f"./data/downloaded/{author}/emmood/"
    for file in os.listdir(path):
        NovelId+=1
        filepath = os.path.join(path, file)
        novel_map.append(file.split('.')[0])
        with open(filepath) as f:
            lines = f.readlines()
            
        for line in lines:
            matches = re.match(pattern, line)

            # Extract the matched groups
            column1 = matches.group(1) # sentence id
            #column2 = matches.group(2)
            column3 = matches.group(3) # emotion labelled by person A
            column4 = matches.group(4) # emotion labelled by person B
            #column5 = matches.group(5) # mood labelled by person 
            #column5 = matches.group(6) # mood labelled by person 
            Text = matches.group(7) #sentence

            SentenceId = column1

            if column3 != 'N': #emoA
                Emotion = column3
            elif column3 == 'N' and column4 != 'N': #emoA
                Emotion = column4
            else:
                Emotion = 'N'
            
            data.append([NovelId, SentenceId, Emotion, Text])

df = pd.DataFrame(data,columns=columns)
df_novel_map = pd.Series(novel_map, name="NovelName")

In [3]:
df["Emotion"].value_counts()

N      9395
H      1883
Sa      988
A       801
F       799
Su-     582
D       491
Su+     363
Name: Emotion, dtype: int64

In [4]:
emo_dict = {'N': "Neutral",
            'H': "Happiness", 
            'Sa': "Sadness",
            'A': "Anger",
            'F': "Fear",
            'Su-': "Surprise",
            'Su+': "Surprise", # both surprise categories (positive & negative are combined)
            'D': "Disgust"}

In [5]:
df.replace({"Emotion": emo_dict}, inplace=True)

In [6]:
df_novel_map.head()

0    106_the_poor_millers_boy_and_the_cat
1           114_the_cunning_little_tailor
2     11_little_brother_and_little_sister
3     13_the_three_little_men_in_the_wood
4                    163_the_glass_coffin
Name: NovelName, dtype: object

In [8]:
df.to_csv("./data/novel_sentences_emotionally_labelled.csv", index=False)
df_novel_map.to_csv("./data/novel_indices.csv", index=True, index_label="NovelId")